In [80]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

In [83]:
file_path = '../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')


# Assign the column names based on the original header in the uploaded file
column_names = [
    "Zimmerzahl", "Kindergarten", "Schule", "S-Bahn", "Miete", "Aufzug", "Lage", "Balkon", "Terrasse", "Moebliert", "Quadratmeter", "Studierende", "Kleinfamilie", "DINK", "Alleinerziehende", "Expatriate", "Rentnerpaar"
]
data = data[column_names]

'''
# Bewohnerkategorie bestimmen
resident_columns = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[resident_columns].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']
'''

column_names = [
    "Zimmerzahl", "Kindergarten", "Schule", "S-Bahn", "Miete", "Aufzug", "Lage", "Balkon", "Terrasse", "Moebliert", "Quadratmeter", "Studierende", "Kleinfamilie", "DINK", "Alleinerziehende", "Expatriate", "Rentnerpaar"
]
data = data[column_names]

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ['Aufzug', 'Balkon', 'Terrasse', "Studierende", "Kleinfamilie", "DINK", "Alleinerziehende", "Expatriate", "Rentnerpaar"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

# Funktion zum Berechnen des Durchschnitts aus einem Wertebereich
def calculate_average_range(range_str):
    if isinstance(range_str, str):
        if "ueber" in range_str:
            # Spezieller Fall "über 120"
            return 120
        elif "bis" in range_str:
            # Spezieller Fall "bis 20"
            return 20
        else:
            # Teile den Wertebereich in zwei Zahlen auf
            start, end = map(int, range_str.split('-'))
            # Berechne den Durchschnitt
            return (start + end) / 2
    return range_str

# Wende die Funktion auf die Spalte "Quadratmeter" an
data['Quadratmeter'] = data['Quadratmeter'].apply(calculate_average_range)

# Funktion zum Berechnen des Durchschnitts aus einem Wertebereich
def get_lage(range_str):
    if isinstance(range_str, str):
        if "Abgelegen" in range_str:
            return 0
        elif "Wohngebiet" in range_str:
            return 0
        elif "Hauptstrasse" in range_str:
            return 1
        elif "Nebenstrasse" in range_str:
            return 2
        elif "Spielstrasse" in range_str:
            return 3
        else:
            return -1
    return range_str

# Wende die Funktion auf die Spalte "Quadratmeter" an
data['Lage'] = data['Lage'].apply(get_lage)

# Kindergarten
def get_kindergarten(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 2
        elif "erreichbar" in range_str:
            return 1
        elif "fern" in range_str:
            return 0
        else:
            return -1
    return range

data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)

# Schule
def get_schule(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 2
        elif "erreichbar" in range_str:
            return 1
        elif "fern" in range_str:
            return 0
    return range

data['Schule'] = data['Schule'].apply(get_schule)

# S-Bahn
def get_bahn(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 4
        elif "erreichbar" in range_str:
            return 3
        elif "mit Bus" in range_str:
            return 2
        elif "fern" in range_str:
            return 1
        elif "nein" in range_str:
            return 0
    return range

data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)

# miete
def calculate_miete(value):
    if '-' in value:
        start, end = value.split('-')
        return (int(start) + int(end)) / 2
    else:
        return int(value)

data['Miete'] = data['Miete'].apply(calculate_miete)

# Zimmer
def kategorisiere_zimmer(zimmer):
    if zimmer in ['Ein Zimmer', '1 Zimmer']:
        return 0
    elif zimmer in ['Zwei Zimmer', 'Drei Zimmer', '2 Zimmer', '3 Zimmer', '1-2 Zimmer', '2-3 Zimmer', ]:
        return 1
    elif zimmer in ['Vier Zimmer', 'Fünf Zimmer', '3-4 Zimmer', '4-5 Zimmer', '4 Zimmer', '5 Zimmer']:
        return 2
    elif zimmer in ['Sechs Zimmer', '5-6 Zimmer', '6 Zimmer']:
        return 3

data['Zimmerzahl'] = data['Zimmerzahl'].apply(kategorisiere_zimmer)

# mobliert
def get_mobliert(range_str):
    if isinstance(range_str, str):
        if "ja" in range_str:
            return 2
        elif "teilmoebliert" in range_str:
            return 1
        elif "nein" in range_str:
            return 0
    return range

data['Moebliert'] = data['Moebliert'].apply(get_mobliert)

print(data.head(20))

    Zimmerzahl  Kindergarten  Schule  S-Bahn  Miete  Aufzug  Lage  Balkon  \
0          1.0             0       1       3  425.5       0     2       0   
1          3.0             1       0       4  825.5       1     1       0   
2          1.0             2       0       4  325.5       0     1       0   
3          1.0             1       0       3  425.5       1     0       1   
4          1.0             2       2       4  475.5       1     1       0   
5          2.0             2       1       4  675.5       0     1       0   
6          1.0             1       0       4  475.5       1     2       1   
7          1.0             2       0       4  525.5       1     1       0   
8          2.0             1       0       3  725.5       1     1       0   
9          1.0             1       0       3  425.5       1     0       0   
10         2.0             0       0       4  525.5       0     1       0   
11         2.0             1       2       4  725.5       0     3       0   

/tmp/ipykernel_21177/4258420215.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [91]:
# Modellierung des Bayes-Netzes
model = BayesianNetwork([
    ("Expatriate", "Moebliert"),
    ("Expatriate", "Lage"),
    ("DINK", "Miete"),
    ("DINK", "Balkon"),
    ("Kleinfamilie", "Zimmerzahl"),
    ("Kleinfamilie", "Kindergarten"),
    ("Alleinerziehende", "Kindergarten"),
    ("Alleinerziehende", "Schule"),
    ("Rentnerpaar", "Terrasse"),
    ("Rentnerpaar", "Aufzug"),
    ("Studierende", "Miete"),
    ("Studierende", "S-Bahn"),
])

# Schritt 4: Training des Bayes-Netzes
model.fit(data, estimator=MaximumLikelihoodEstimator)


# Schritt 5: Inferenz durchführen
inference = VariableElimination(model)

# Beispiel-Eingabe: Teilweise bekannte Daten
example_input = {
    "Zimmerzahl": data['Zimmerzahl'].unique()[1],  # Verwende korrekte Zustandsnummer
    "Miete": data['Miete'].unique()[0],           # Verwende korrekte Zustandsnummer
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.map_query(variables=["Alleinerziehende", "Expatriate", "Rentnerpaar", "Studierende", "DINK", "Kleinfamilie"], evidence=example_input)

print("Klassifikationsergebnis:")
print(prediction)


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

Klassifikationsergebnis:
{'Expatriate': np.int64(0), 'Alleinerziehende': np.int64(0), 'Studierende': np.int64(0), 'Kleinfamilie': np.int64(0), 'DINK': np.int64(0), 'Rentnerpaar': np.int64(0)}
